In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("TP4_SparkMLlib").getOrCreate()


**1 : Charger les fichiers**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2010-12-01.csv to 2010-12-01.csv
Saving 2010-12-02.csv to 2010-12-02.csv
Saving 2010-12-03.csv to 2010-12-03.csv
Saving 2010-12-05.csv to 2010-12-05.csv
Saving 2010-12-06.csv to 2010-12-06.csv
Saving 2010-12-07.csv to 2010-12-07.csv
Saving 2010-12-08.csv to 2010-12-08.csv
Saving 2010-12-09.csv to 2010-12-09.csv
Saving 2010-12-10.csv to 2010-12-10.csv
Saving 2010-12-12.csv to 2010-12-12.csv
Saving 2010-12-13.csv to 2010-12-13.csv
Saving 2010-12-14.csv to 2010-12-14.csv
Saving 2010-12-15.csv to 2010-12-15.csv
Saving 2010-12-16.csv to 2010-12-16.csv
Saving 2010-12-17.csv to 2010-12-17.csv
Saving 2010-12-19.csv to 2010-12-19.csv
Saving 2010-12-20.csv to 2010-12-20.csv
Saving 2010-12-21.csv to 2010-12-21.csv
Saving 2010-12-22.csv to 2010-12-22.csv
Saving 2010-12-23.csv to 2010-12-23.csv
Saving 2011-01-04.csv to 2011-01-04.csv
Saving 2011-01-05.csv to 2011-01-05.csv


**- charger les données**

In [ ]:
# Charger les fichiers du dossier tp4_data dans un DataFrame Spark
df = spark.read.csv("/content/*.csv", header=True, inferSchema=True)

# Afficher les premières lignes
df.show(5)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



**2: Afficher le schéma du DataFrame**

In [ ]:
# Afficher le schéma du DataFrame
df.printSchema()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



**3: Remplir les valeurs manquantes avec 0**

In [ ]:
# Remplir les valeurs manquantes (NaN) avec la valeur 0
df = df.fillna(0)


**4 : Ajouter la colonne "day_of_week"**

In [ ]:
from pyspark.sql.functions import dayofweek, to_date , date_format

# Convertir la colonne InvoiceDate en format date
df = df.withColumn("InvoiceDate", to_date(df["InvoiceDate"], "yyyy-MM-dd"))
df.show(5)


# Ajouter une nouvelle colonne "day_of_week" représentant le jour de la semaine
#df = df.withColumn("day_of_week", dayofweek("InvoiceDate"))
df = df.withColumn("day_of_week", date_format("InvoiceDate", "EEEE"))

df.select("day_of_week").distinct().show()


+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6| 2010-12-06|     2.95|   15987.0|United Kingdom|     Monday|
|   537226|    21713|CITRONELLA CANDLE...|       8| 2010-12-06|      2.1|   15987.0|United Kingdom|     Monday|
|   537226|    22927|GREEN GIANT GARDE...|       2| 2010-12-06|     5.95|   15987.0|United Kingdom|     Monday|
|   537226|    20802|SMALL GLASS SUNDA...|       6| 2010-12-06|     1.65|   15987.0|United Kingdom|     Monday|
|   537226|    22052|VINTAGE CARAVAN G...|      25| 2010-12-06|     0.42|   15987.0|United Kingdom|     Monday|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----

**5: Diviser les données en ensemble d'apprentissage et de test**

In [ ]:
# Séparer les données selon la date
training_df = df.filter(df["InvoiceDate"] < "2010-12-13")
test_df = df.filter(df["InvoiceDate"] >= "2010-12-13")


**6: Créer un StringIndexer**
**7: Discussion sur l'ordre implicite des jours**
L'utilisation de StringIndexer crée une numérotation arbitraire pour les jours de la semaine. Pour éviter l'ordre implicite (exemple : 6 pour Samedi et 1 pour Lundi), il est possible d'utiliser OneHotEncoder ou VectorAssembler pour transformer les jours en vecteurs d'indicateurs.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Créer un StringIndexer pour la colonne "day_of_week"
indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_indexed")
index_df = indexer.fit(df).transform(df)
# Afficher pour vérifier que day_of_week_indexed est créé
index_df.select("InvoiceDate", "day_of_week", "day_of_week_indexed").show(5)

# Étape 2 : Appliquer OneHotEncoder sur day_of_week_indexed
encoder = OneHotEncoder(inputCol="day_of_week_indexed", outputCol="day_of_week_encoded")
df_encoded = encoder.fit(index_df).transform(index_df)


# Afficher le DataFrame final avec les colonnes encodées
df_encoded.select("InvoiceDate", "day_of_week", "day_of_week_indexed", "day_of_week_encoded").show(5)

+-----------+-----------+-------------------+
|InvoiceDate|day_of_week|day_of_week_indexed|
+-----------+-----------+-------------------+
| 2010-12-06|     Monday|                2.0|
| 2010-12-06|     Monday|                2.0|
| 2010-12-06|     Monday|                2.0|
| 2010-12-06|     Monday|                2.0|
| 2010-12-06|     Monday|                2.0|
+-----------+-----------+-------------------+
only showing top 5 rows

+-----------+-----------+-------------------+-------------------+
|InvoiceDate|day_of_week|day_of_week_indexed|day_of_week_encoded|
+-----------+-----------+-------------------+-------------------+
| 2010-12-06|     Monday|                2.0|      (5,[2],[1.0])|
| 2010-12-06|     Monday|                2.0|      (5,[2],[1.0])|
| 2010-12-06|     Monday|                2.0|      (5,[2],[1.0])|
| 2010-12-06|     Monday|                2.0|      (5,[2],[1.0])|
| 2010-12-06|     Monday|                2.0|      (5,[2],[1.0])|
+-----------+-----------+--------

**9: Créer un VectorAssembler**

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Créer le VectorAssembler pour combiner UnitPrice, Quantity, et day_of_week_encoded
assembler = VectorAssembler(inputCols=["UnitPrice", "Quantity", "day_of_week_encoded"], outputCol="features")
df_final = assembler.transform(df_encoded)

# Afficher les résultats
df_final.select("UnitPrice", "Quantity", "day_of_week_encoded", "features").show(5)

+---------+--------+-------------------+--------------------+
|UnitPrice|Quantity|day_of_week_encoded|            features|
+---------+--------+-------------------+--------------------+
|     2.95|       6|      (5,[2],[1.0])|(7,[0,1,4],[2.95,...|
|      2.1|       8|      (5,[2],[1.0])|(7,[0,1,4],[2.1,8...|
|     5.95|       2|      (5,[2],[1.0])|(7,[0,1,4],[5.95,...|
|     1.65|       6|      (5,[2],[1.0])|(7,[0,1,4],[1.65,...|
|     0.42|      25|      (5,[2],[1.0])|(7,[0,1,4],[0.42,...|
+---------+--------+-------------------+--------------------+
only showing top 5 rows



**10: Créer un pipeline avec les étapes précédentes**

In [ ]:
from pyspark.ml import Pipeline

# Créer un pipeline avec indexer et assembler
pipeline = Pipeline(stages=[indexer, encoder, assembler])


**11: Indexer des valeurs uniques**

---


StringIndexer gère automatiquement le nombre de valeurs uniques, donc il n'est pas nécessaire de faire un traitement supplémentaire. La méthode .fit() entraîne l'indexeur en tenant compte des valeurs uniques de la colonne.

**12: Transformer les données d'apprentissage**

In [ ]:
# Ajuster et transformer les données d'apprentissage
model = pipeline.fit(training_df)
df_train_transformed = model.transform(training_df)
df_train_transformed.show(5)


+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+-------------------+-------------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_indexed|day_of_week_encoded|            features|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----------+-------------------+-------------------+--------------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6| 2010-12-06|     2.95|   15987.0|United Kingdom|     Monday|                4.0|      (5,[4],[1.0])|(7,[0,1,6],[2.95,...|
|   537226|    21713|CITRONELLA CANDLE...|       8| 2010-12-06|      2.1|   15987.0|United Kingdom|     Monday|                4.0|      (5,[4],[1.0])|(7,[0,1,6],[2.1,8...|
|   537226|    22927|GREEN GIANT GARDE...|       2| 2010-12-06|     5.95|   15987.0|United Kingdom|     Monday|                4.0|    

**13 : Créer une instance de KMeans**

In [ ]:
from pyspark.ml.clustering import KMeans

# Créer une instance de KMeans avec 20 clusters
kmeans = KMeans(k=20, featuresCol="features", seed=42)


**14 : Lancer l'apprentissage de KMeans**

In [ ]:
# Entraîner le modèle KMeans
kmeans_model = kmeans.fit(df_train_transformed)


15 : Prédire sur l'ensemble de test



In [ ]:
# Transformer les données de test en utilisant le pipeline
test_transformed = model.transform(test_df)

# Effectuer des prédictions sur l'ensemble de test
predictions = kmeans_model.transform(test_transformed)


16 : Calculer le coefficient de silhouette

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Calculer le coefficient de silhouette
evaluator = ClusteringEvaluator(predictionCol="prediction", featuresCol="features", metricName="silhouette")
silhouette = evaluator.evaluate(predictions)
print(f"Coefficient de silhouette : {silhouette}")


Coefficient de silhouette : 0.744526969120679
